In [1]:
# import packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException
import time 
import os
import json
import urllib


In [2]:
# Some urls are blocked from untrusted sites, the below code make python environment to trust every site
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
def download_images():
    
    f = open('site_urls.txt','r')
    search_url = f.readlines()[12]
    print(search_url)
   
    # path of chorme drive to run selenium
    path = r' chromedriver'
    
    driver = webdriver.Chrome(path)
    
    k = open('wildlife keywords.txt','r')
    kw = k.readlines()
    
    for i in range(0,len(kw)):
        print(i)
        keyW = kw[i]
        
        driver.get(search_url)

        time.sleep(2)
        # search text box 
        box = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[2]/div[1]/div[2]/div[3]/form/div[1]/div[2]/input')  
        box.clear()
        
        #sending the keyword
        box.send_keys(keyW)
        time.sleep(2)
        
        # try block if there are no ad results for the keyword it will continue with next keyword
        try:
            # collecting links of ad results
            links = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div[3]/div/div[2]/div[3]/div[2]')
            sub_lnks = links.find_elements_by_tag_name('a')

            lnks_lst = []

            for lnk in sub_lnks:
                lnks_lst.append(lnk.get_attribute('href'))
            print(lnks_lst[0:3])

            # making a directory like URL+Keyword_number , example 12_0 , 12_1,...
            if lnks_lst:
                if not os.path.exists(str(12)+'_'+str(i)):
                    os.mkdir(str(12)+'_'+str(i))
            #  iterating through ads results    
            for j in range(1,3):
                driver.get(lnks_lst[j])
                print(lnks_lst[j])
                time.sleep(2)
                # Title capturing
                sub_txt = driver.find_element_by_xpath('//*[@id="product_addtocart_form"]/div[5]/h2').text
                
                #making directory inside the above created directory like 0 means first ad, 1 for second ad....
                if not os.path.exists(str(12)+'_'+str(i) +'/'+ str(j)):
                    os.mkdir(str(12)+'_'+str(i) +'/'+ str(j))
                    
                # making dictionary
                details = {
                    'name' : sub_txt,
                    'price' : driver.find_element_by_xpath('/html/body/div/div[1]/div[3]/div/div/div[3]/div[1]/form/div[7]/div/div[2]').get_attribute("innerText"),
                    'location': None,
                    'details': driver.find_element_by_xpath('//*[@id="product_addtocart_form"]/div[7]/div/ul').get_attribute("innerText"),
                    'description' : driver.find_element_by_xpath('//*[@id="product_tabs_description_contents"]/div[1]/div').get_attribute("innerText"),
                    'url' : lnks_lst[j]
                }
                
                # writing the dictionary to text file like 0.txt for first ad ....
                with open(str(12)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '.txt', 'w') as tfile:
                     tfile.write(json.dumps(details))
                # collecting image urls
                image_lst = []
                elem1 = driver.find_element_by_xpath('//*[@id="product_addtocart_form"]/div[6]/div/div[2]/div')
                sub = elem1.find_elements_by_tag_name('a')
                for k,l in enumerate(sub):
                    href = l.get_attribute('href')
                    image_lst.append(href)
                    print(href)
                    # collecting images and saving it inside the above created directory like 0_0.jpg for first ad first image
                    # 0_1.jpg for first ad second image ...
                    if href != None:
                        urllib.request.urlretrieve(str(href), os.path.join(str(12)+'_'+str(i), str(j), str(j)+'_'+str(k)+'.jpg'))
                # writing image urls to text file like 0_imgs.txt for first ad ....
                with open(str(12)+'_'+str(i) +'/'+ str(j) + '/' + str(j) + '_imgs.txt', 'w') as tfile:
                     tfile.write(json.dumps(image_lst)) 
        except:
            continue
            
    driver.close()
    

In [7]:
def main():
    download_images()
    
if __name__ == "__main__":
    main()

http://www.thetaxidermystore.com/catalogsearch/result/index/?cat=0&dir=desc&limit=60&order=sku&q=tiger

0
['https://www.thetaxidermystore.com/alligator-replica-taxidermy-mount-for-sale-27004.html', 'https://www.thetaxidermystore.com/alligator-replica-taxidermy-mount-for-sale-27004.html', 'https://www.thetaxidermystore.com/alligator-replica-taxidermy-mount-for-sale-26016.html']
https://www.thetaxidermystore.com/alligator-replica-taxidermy-mount-for-sale-27004.html
https://www.thetaxidermystore.com/media/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/a/l/alligator_replica_mount_for_sale_27004_the_taxidermy_store.jpeg
https://www.thetaxidermystore.com/media/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/2/7/27004_-_2_of_8.jpeg
https://www.thetaxidermystore.com/media/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/2/7/27004_-_3_of_8.jpeg
https://www.thetaxidermystore.com/media/catalog/product/cache/1/image/9df78eab33525d08d6e5fb8d27136e95/2/7/